In [1]:
import pathlib
import pandas as pd

In [2]:
filename = pathlib.Path.cwd() / 'Base_MindMiners.xlsx'
db = pd.read_excel('Base_MindMiners.xlsx')

In [3]:
db.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,Variável,Tipo,Pergunta,Respostas/Categorias
1,NaN,P1,Resposta Única,Quantas vezes você costuma viajar por ano?,(1) 1 a 3 vezes; (2) 4 a 6 vezes; (3) 7 a 10 v...
2,NaN,P2,Resposta Múltipla,Qual foi o princípal motivo de suas ultimas vi...,(1) Visitar amigos; (2) Visitar familia; (3)La...
3,NaN,P3,Escala,O quão importante para você é a infraestrutura...,1 (Não Utilizo) ; 2 ; 3 ; 4 ; 5 ; 6 ; 7 (Utili...
4,NaN,P4,Escala,Restaurante,1 (Não utilizo) ; 2 ; 3 ; 4 ; 5 ; 6 ; 7 (Utili...


In [4]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  0 non-null      float64
 1   Unnamed: 1  72 non-null     object 
 2   Unnamed: 2  72 non-null     object 
 3   Unnamed: 3  72 non-null     object 
 4   Unnamed: 4  72 non-null     object 
dtypes: float64(1), object(4)
memory usage: 2.9+ KB


In [5]:
df = pd.read_excel('Base_MindMiners.xlsx', sheet_name='Pesquisa-Respostas')

In [6]:
df.head()

,Unnamed: 0,P1,P2,P3,P4,Unnamed: 5,P5,P6,.P7,P8,...,Qtipcama,Qcama,Qest,Qche,Qocup,Qid,Qgen,Qreg,Qclasse,Qs
0,NaN,2.0,7,7.0,5.0,NaN,4,2,6,7.0,...,Viuvo,1.0,4.0,Sábado,2.0,6.0,1.0,2.0,B,Dados para Marketing
1,NaN,3.0,2,2.0,6.0,4.0,4,2,5,1.0,...,Solteiro,1.0,1.0,Quinta-Feira,2.0,6.0,1.0,5.0,B,NaN
2,NaN,1.0,7,7.0,5.0,NaN,4,5,6,6.0,...,Viuvo,1.0,3.0,Sábado,3.0,4.0,1.0,2.0,B,Dados para Hotel
3,NaN,1.0,3,4.0,4.0,NaN,6,6,1,7.0,...,Casado,1.0,6.0,Terça-Feira,3.0,1.0,2.0,2.0,B,Marketing
4,NaN,2.0,7,6.0,7.0,NaN,3,3,7,6.0,...,Casado,1.0,4.0,Quinta-Feira,6.0,2.0,1.0,2.0,A,Dados para Hotel


In [7]:
# Remover as colunas com índices 'Unnamed'
df = df.drop(columns=[coluna for coluna in df.columns if coluna.startswith('Unnamed:')])

In [8]:
df.head()

,P1,P2,P3,P4,P5,P6,.P7,P8,P9,P10,...,Qtipcama,Qcama,Qest,Qche,Qocup,Qid,Qgen,Qreg,Qclasse,Qs
0,2.0,7,7.0,5.0,4,2,6,7.0,5,1.0,...,Viuvo,1.0,4.0,Sábado,2.0,6.0,1.0,2.0,B,Dados para Marketing
1,3.0,2,2.0,6.0,4,2,5,1.0,3,2.0,...,Solteiro,1.0,1.0,Quinta-Feira,2.0,6.0,1.0,5.0,B,NaN
2,1.0,7,7.0,5.0,4,5,6,6.0,5,2.0,...,Viuvo,1.0,3.0,Sábado,3.0,4.0,1.0,2.0,B,Dados para Hotel
3,1.0,3,4.0,4.0,6,6,1,7.0,6,6.0,...,Casado,1.0,6.0,Terça-Feira,3.0,1.0,2.0,2.0,B,Marketing
4,2.0,7,6.0,7.0,3,3,7,6.0,5,2.0,...,Casado,1.0,4.0,Quinta-Feira,6.0,2.0,1.0,2.0,A,Dados para Hotel


In [9]:
# Usando a função split para remover espaços nos indexes das colunas
df.columns = df.columns.str.strip()

In [10]:
# # Usando a função str.replace para remover "." dos indexes das colunas
df.columns = df.columns.str.replace(".", "")

<ipython-input-10-99a3ba088efe>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace(".", "")


Removendo conteudo indesejado das colunas

In [11]:
# Lista com todas as colunas
nomes = list(df)
nomes

['P1',
 'P2',
 'P3',
 'P4',
 'P5',
 'P6',
 'P7',
 'P8',
 'P9',
 'P10',
 'P11',
 'P12',
 'P13',
 'P14',
 'P15',
 'P16',
 'P17',
 'P18',
 'P19',
 'P20',
 'P21',
 'P22',
 'P23',
 'P24',
 'P25',
 'P26',
 'P27',
 'P28',
 'P29',
 'P30',
 'P31',
 'P32',
 'P33',
 'P34',
 'P35',
 'P36',
 'P37',
 'P38',
 'P39',
 'P40',
 'P41',
 'P42',
 'P43',
 'P44',
 'P45',
 'P46',
 'P47',
 'P48',
 'P49',
 'P50',
 'P51',
 'P52',
 'P53',
 'P54',
 'P55',
 'P56',
 'P57',
 'P58',
 'Qesc',
 'Qciv',
 'Qnat',
 'Qtipcama',
 'Qcama',
 'Qest',
 'Qche',
 'Qocup',
 'Qid',
 'Qgen',
 'Qreg',
 'Qclasse',
 'Qs']

In [12]:
# Primeria coluna
df.loc[~df.loc[:, nomes[0]].isin(range(1,5)), nomes[0]] = 'nan'

In [13]:
# Seguna coluna
df.loc[~df.loc[:, nomes[1]].isin(range(1,9)), nomes[1]] = 'nan'

In [14]:
# Coluna 3 a 58
for coluna in range(2, 58):
    df.loc[~df.loc[:, nomes[coluna]].isin(range(1,8)), nomes[coluna]] = 'nan'

In [15]:
# Coluna Qesc
df.loc[~df.loc[:, nomes[58]].isin(range(1,7)), nomes[58]] = 'nan'

In [18]:
# Coluna Qcama
df.loc[~df.loc[:, nomes[62]].isin(range(1,5)), nomes[62]] = 'nan'

In [20]:
# Coluna Qche
lista_quinta_feira = ["Quinta-Feira", "Quinta Feira", "quinta-Feira", "quintaFeira"]
df.loc[df.loc[:, nomes[64]].isin(lista_quinta_feira), nomes[64]] = 'Quinta-Feira'

In [33]:
tab = df['P2'].value_counts(sort=False)
print('Tabela de frequências absolutas - SATISFAÇÃO')
tab.to_frame()

Tabela de frequências absolutas - SATISFAÇÃO


,P2
7,219
2,182
3,416
1,26
nan,276
5,123
4,14
6,12


In [22]:
# Checando qual indice pertence a qual coluna
nomes[65]

'Qocup'

# Essa célula abaixo não está pronta ainda e n consegui acabar ainda

In [ ]:
# criar lista com as strings 'P1', 'P2', 'P3', ..., 'P58'
cols_to_numeric = []
for i in range(1, 59):
    cols_to_numeric.append('P' + str(i))

    
# aplicar pd.to_numeric a cada coluna
for col in cols_to_numeric:
    df.columns[col] = pd.to_numeric(df.columns[col], errors='coerce')
    
